In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Пути к файлам данных
path_to_dataset = 'C:\\Users\\Pilyay Andrey\\Downloads\\Датасет. Газпромбанк\\Dataset_Yakutia.xlsx'
path_to_products = 'C:\\Users\\Pilyay Andrey\\Downloads\\Датасет. Газпромбанк\\Products.xlsx'

# Загрузка данных
df = pd.read_excel(path_to_dataset, na_values='NULL')
products_df = pd.read_excel(path_to_products)

# Преобразование числовых столбцов
numeric_cols = ['age', 'cnt_tr_all_3m', 'cnt_tr_top_up_3m', 'cnt_tr_cash_3m', 'cnt_tr_buy_3m', 'cnt_tr_mobile_3m', 
                'cnt_tr_oil_3m', 'cnt_tr_on_card_3m', 'cnt_tr_service_3m', 'cnt_zp_12m', 'sum_zp_12m', 
                'limit_exchange_count', 'max_outstanding_amount_6m', 'avg_outstanding_amount_3m', 
                'cnt_dep_act', 'sum_dep_now', 'avg_dep_avg_balance_1month', 'max_dep_avg_balance_3month', 
                'qnt_months_from_last_visit']
for col in numeric_cols:
    df[col] = df[col].astype(str).str.replace(',', '.').astype(float)

# Обработка отсутствующих значений
imputer = SimpleImputer(strategy='constant', fill_value=0)
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

# Преобразование категориальных данных в строки
categorical_features = ['gender', 'reg_region_nm', 'app_vehicle_ind', 'app_position_type_nm', 'visit_purposes']
for col in categorical_features:
    df[col] = df[col].astype(str)

# Трансформация признаков
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Подготовка данных
X = df.drop('super_clust', axis=1)
y = df['super_clust']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Применение трансформера к данным
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Определение параметров для Grid Search
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20]
}
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train_transformed, y_train)

# Лучшие параметры и модель
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Предсказания и оценка качества модели
predictions = best_model.predict(X_test_transformed)
accuracy = accuracy_score(y_test, predictions)
print(f'Best Accuracy: {accuracy}')
print(f'Best Parameters: {best_params}')

# Загрузка и настройка модели GPT-2
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model_gpt2 = GPT2LMHeadModel.from_pretrained('gpt2')

# Функция для генерации персонализированного маркетингового сообщения
def generate_personalized_marketing_message(user_data, model_rf, model_gpt2, tokenizer, preprocessor):
    # Преобразование данных пользователя с помощью preprocessor
    user_data_transformed = preprocessor.transform(user_data)

    # Предсказание идентификатора пользователя и уверенности модели
    user_prediction = model_rf.predict(user_data_transformed)
    user_confidence = model_rf.predict_proba(user_data_transformed).max()


    # Генерация текста коммуникации
    prompt = "Уважаемый клиент, мы рады предложить вам уникальные условия по автокредиту. Вот что мы можем вам предложить: выгодные ставки, быстрое одобрение и специальные условия для постоянных клиентов."
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    generated_output = model_gpt2.generate(input_ids, max_length=200, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    marketing_message = tokenizer.decode(generated_output[0], skip_special_tokens=True)

    return user_prediction[0], marketing_message, user_confidence

# Пример использования
user_data = X_test.iloc[0:1]  # Пример данных пользователя для генерации сообщения
user_id, marketing_message, confidence = generate_personalized_marketing_message(user_data, best_model, model_gpt2, tokenizer, preprocessor)
print(f"User ID: {user_id}, Confidence: {confidence}")
print("Marketing Message:", marketing_message)


Fitting 3 folds for each of 4 candidates, totalling 12 fits
Best Accuracy: 0.645
Best Parameters: {'max_depth': 10, 'n_estimators': 100}
User ID: c. Масс-ЗП закредитованные (12), Confidence: 0.2860198104909109
Marketing Message: Уважаемый клиент, мы рады предложить вам уникальные условия по автокредиту. Вот что мы можем вам предложить: выгодные ставки, быстрое одобрение и специальные условия для постоянных клиентов. В
